In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

import warnings
warnings.filterwarnings("ignore")
import random

import dtw

numberOfTimeStep = 14

## Features

In [ ]:
folders = ["s1", "s2", "s3", "s4"]

keys = ['neighbor_QUI', 'neighbor_CAR', 'neighbor_PAP', 'neighbor_OXA', 
        'numberOfPatients', 'neighbor_CF3', 'neighbor_PEN']

binary = []

continuous =  [variable for variable in keys if variable not in binary]

## Correlations between variables
- B-B: Phi 
- C-C: Pearson
- B-C: Point Biserial

In [ ]:
for c in range(len(folders)):

    ########################
    ###### PARAMETERS ######
    ########################
    numberOfTimeStep = 14
    ########################

    ####### BEGIN CODE ===>>
    # Load data
    X_train = np.load("../../../../../../ORIGINAL_DATA/MDR/splits_14_days/cmi/split_" + str(c) +
                          "/X_train_tensor_" + str(0)+ ".npy")


    X_val = np.load("../../../../../../ORIGINAL_DATA/MDR/splits_14_days/cmi/split_" + str(c) +
                        "/X_val_tensor_" + str(0)+ ".npy")
    X_train = np.concatenate((X_train, X_val), axis=0)
    
    # Reshape data by features --> PxTxF to FxTxP
    X_train = dtw.reshape_patients_by_features(X_train, keys, numberOfTimeStep)
    
    print("X_train shape after reshape:", X_train.shape)
    
    # Compute Heterogeneous Gower Distance
    result = Parallel(n_jobs=-1)(
        delayed(dtw.compute_hgd_matrix)(X_train, X_train[f1, :, :], X_train[f2, :, :], keys[f1], keys[f2], binary, continuous)
        for f1 in range(X_train.shape[0]) for f2 in range(X_train.shape[0])
    )
    
    # Replace NaNs with zero
    lista_sin_nan = dtw.replace_nan_with_zero(result)
    
    # Compute DTW distance based on HGD matrix
    dtw_matrix_train = Parallel(n_jobs=96)(
        delayed(dtw.compute_new_dtw)(lista_sin_nan[i])
        for i in range(X_train.shape[0] * X_train.shape[0])
    )

    data = np.array(dtw_matrix_train).reshape((X_train.shape[0], X_train.shape[0]))
    
    # Save the results as CSV
    pd.DataFrame(data, columns=keys, index=keys).to_csv("./estimatedGraphs/dtw/" + folders[c] + "/X_train_allMTS"+ ".csv", index=False)
